<a href="https://colab.research.google.com/github/NumanAloko/ML-for-CFS-Built-up-Columns/blob/main/Regression%20ML%20Models/XGBoost_ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Training and testing of XGBoost ML model**

In [ ]:
# Note that scikit-learn version 1.5 is compatible with the XGBoost regressor; ensure that the version is below 1.6. If a higher version of scikit-learn is installed,  uninstall it.
!pip uninstall -y scikit-learn

In [ ]:
# Install the compatible version with XGBoost
!pip install scikit-learn==1.5.0


In [ ]:
########## Import necessary libraries ########################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
############################### Create Dataframe ###############################

# Load the dataset
csv_url= "https://raw.githubusercontent.com/NumanAloko/ML-for-CFS-Built-up-Columns/refs/heads/main/CFS_Built-up_Columns_ML_Dataset.csv"
df = pd.read_csv(csv_url, header=0)
df = df.dropna(how='all').dropna(axis=1, how='all')

data_x = df[['L', 't', 'h', 'b','KL_r','Fy', 'λc', 'λ(le-d)']]  # Input Features
y = df[['Pt/Py']]                                               # Target Values

# Data Scaling
sc_X = StandardScaler()
sc_y = StandardScaler()
X_sc = sc_X.fit_transform(data_x)
y_sc = sc_y.fit_transform(y)

# Create DataFrames with the standardised data and keep track of indices
X_sc_df = pd.DataFrame(X_sc, columns=data_x.columns, index=data_x.index)
y_sc_df = pd.DataFrame(y_sc, columns=['Pt/Py'], index=y.index)

# Split the data into training and test sets by the ratio of 70:30.

X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    X_sc_df, y_sc_df, X_sc_df.index, test_size=0.3, random_state=123)

In [ ]:
# Define the XGBoost model
xgb = XGBRegressor(verbosity=0, early_stopping_rounds=10)

# Hyperparameter tuning based on Grid search method
# Define the parameter grid
#param_grid = {
    #'n_estimators': [100, 200, 300, 500],
    #'learning_rate': [0.01, 0.05, 0.1, 0.2],
    #'max_depth': [3, 6, 9, 12],
    #'subsample': [0.5, 0.7, 0.9, 1.0],
    #'colsample_bytree': [0.5, 0.7, 0.9, 1.0],
    #'reg_alpha': [0, 0.1, 0.5, 1],  # L1 regularization
    #'reg_lambda': [1, 1.5, 2, 3],   # L2 regularization
    #'objective': ['reg:squarederror']
#}

# Optimised Hyperparameters (Best parameters selected after tuning)
param_grid = {
    'n_estimators': [500],
    'learning_rate': [0.2],
    'max_depth': [3],
    'subsample': [1.0],
    'colsample_bytree': [0.7],
    'reg_alpha': [0.5],  # L1 regularization
    'reg_lambda': [3],   # L2 regularization
    'objective': ['reg:squarederror']
}

#Initialise GridSearchCV
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search
grid_search.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

# Get the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
 # Make predictions on the training and test sets
y_pred_xgb_train = grid_search.predict(X_train)
y_pred_xgb_test = grid_search.predict(X_test)

# De-normalise predictions
y_pred_xgb_train_de = pd.DataFrame(y_pred_xgb_train * sc_y.scale_ + sc_y.mean_, columns=y_train.columns, index=y_train.index)
y_pred_xgb_test_de = pd.DataFrame(y_pred_xgb_test * sc_y.scale_ + sc_y.mean_, columns=y_test.columns, index=y_test.index)

# Combine again all true target values and predictions of XGBoost model
y_all_de = pd.concat([y.loc[train_indices], y.loc[test_indices]])
y_pred_all_de = pd.concat([y_pred_xgb_train_de, y_pred_xgb_test_de])

In [ ]:
#########################################  Performance Indicators ########################################

# Coefficient of determination (r2)
r2_train = r2_score(y.loc[train_indices], y_pred_xgb_train_de)
r2_test = r2_score(y.loc[test_indices], y_pred_xgb_test_de)
r2_all = r2_score(y_all_de, y_pred_all_de)

# Calculation of RMSE (Root mean squared error) for training data
rmse_train = mean_squared_error(y.loc[train_indices], y_pred_xgb_train_de, squared=False)
# Calculation of RMSE  (Root mean squared error) for test data
rmse_test = mean_squared_error(y.loc[test_indices], y_pred_xgb_test_de, squared=False)
# Calculation of RMSE  (Root mean squared error)for all data
rmse_all = mean_squared_error(y_all_de, y_pred_all_de, squared=False)

print(f'r2 score for training = {r2_train:.4f}')
print(f'r2 score for testing = {r2_test:.4f}')
print(f'r2 score for all = {r2_all:.4f}')

print(f'RMSE score for training = {rmse_train:.4f}')
print(f'RMSE score for testing = {rmse_test:.4f}')
print(f'RMSE score for all = {rmse_all:.4f}')

# # Calculation of ratios of true target values to XGBoost ML model's predicted values
ratio_train_de = y.loc[train_indices] / y_pred_xgb_train_de
ratio_test_de = y.loc[test_indices] / y_pred_xgb_test_de
ratio_all_de = y_all_de / y_pred_all_de

# Mean of ratios
mean_ratio_train_de = ratio_train_de.mean(axis=0)
mean_ratio_test_de = ratio_test_de.mean(axis=0)
mean_ratio_all_de = ratio_all_de.mean(axis=0)

# Standard deviation of ratios
std_ratio_train_de = ratio_train_de.std(axis=0)
std_ratio_test_de = ratio_test_de.std(axis=0)
std_ratio_all_de = ratio_all_de.std(axis=0)

print(f'Mean of ratio for training (de-normalized) = {mean_ratio_train_de.mean():.4f}')
print(f'Mean of ratio for testing (de-normalized) = {mean_ratio_test_de.mean():.4f}')
print(f'Mean of ratio for all data (de-normalized) = {mean_ratio_all_de.mean():.4f}')

print(f'Standard deviation of ratio for training (de-normalized) = {std_ratio_train_de.mean():.4f}')
print(f'Standard deviation of ratio for testing (de-normalized) = {std_ratio_test_de.mean():.4f}')
print(f'Standard deviation of ratio for all data (de-normalized) = {std_ratio_all_de.mean():.4f}')

**Make new prediction after training of XGBoost ML Model**

In [ ]:
# Make new prediction
# Manually input the new data
new_data = {
    'L': [1500],               # Length(mm)
    't': [1.5],                # Thickness of the section (mm)
    'h': [175],                # Height of the single section (mm)
    'b': [65],                 # Flange of the single section (mm)
    'KL_r': [25.27],           # Non-dimensioanl member slenderness
    'Fy': [450],               # Yield stress
    'λc': [0.38],              # Global Slenderness
    'λ(le-d)': [2.36],         # Sectional Slenderness
    'A': [988]                 # Area
}

# Create a DataFrame for the new data
new_df = pd.DataFrame(new_data)

area = new_df['A'].values[0]
fy = new_df['Fy'].values[0]

new_df_features = new_df.drop(columns=['A'])

# Scale the new data using the same scaler used for training data
new_data_x_scaled = sc_X.transform(new_df_features)

# Make predictions on the new data
new_predictions_scaled = grid_search.predict(new_data_x_scaled)

# Reverse the scaling on the predictions
new_predictions = new_predictions_scaled * sc_y.scale_ + sc_y.mean_

# Multiply the prediction (Pt/Py) by yielding strength(area x yield stress) of Section area and yield strength
new_predictions = new_predictions * (area * fy *0.001)

# Create a DataFrame for the final predictions
new_predictions_df = pd.DataFrame(new_predictions, columns=['Predicted'])

# Print the final predictions
print("Predicted Axial Strength:", new_predictions_df['Predicted'].values)

**Interpretion of XGBoost ML model's prediction for new instance**

In [ ]:
# Interpretation of New Instance
import shap

# Initialise the explainer and compute SHAP values for the new input
explainer = shap.KernelExplainer(grid_search.predict, shap.sample(X_train, 725))
new_shap_values = explainer.shap_values(new_data_x_scaled)

# Convert the new input data to a DataFrame with the new feature names
new_feature_names = ['L', 't', 'h', 'b', 'KL/r', '$F_y$', '$\\lambda_c$', '$\\lambda_{(le,d)}$']
new_input_df = pd.DataFrame(new_data_x_scaled, columns=new_feature_names)

# Set the figure size
plt.figure(figsize=(8, 6))

# The decision plot for the new instance
shap.decision_plot(explainer.expected_value, new_shap_values[0], new_input_df.iloc[0], feature_names=new_feature_names, highlight=0, show=False)

# Get the current figure and axes
fig = plt.gcf()
ax = plt.gca()

# Set the x limits and the y label
#ax.set_xlim(-1.8, 1.0)
ax.set_ylabel('Input Features', fontsize=14)
ax.set_title('Interpretation of new instance', fontsize=14)

# Add small ticks on the y-axis
ax.tick_params(axis='y', direction='out', length=6, width=2, colors='black')

# Save the figure
ax.grid(False)
plt.show()